Weighting sentences by their position in the review:

We calculate feature importance off of the test set's features because only the test set's sentences get scored (probssentences). Is this legal?

In [16]:
import pickle
import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [17]:
probssentences = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/probssentences.p', 'rb'))
test1 = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/test1.p', 'rb'))

In [18]:
#append doc value to test1
test1['doc'] = [ i for i in range(len(test1))]
print test1.shape
print probssentences.shape

(12500, 4)
(133515, 3)


In [19]:
test1[0:10]

,id,sentiment,review,doc
21130,"""5706_2""",0,"""This movie is like the thousand \""cat and mou...",0
3404,"""11132_3""",0,"""A documentarist, like any filmmaker, must con...",1
14560,"""10770_7""",1,"""With a well thought out cast, this movie was ...",2
6671,"""1058_10""",1,"""this is the best sci-fi that I have seen in m...",3
10982,"""12239_1""",0,"""Ever since I was eight years old I have been ...",4
5526,"""966_7""",1,"""2005 Toronto Film Festival Report It is offic...",5
19301,"""11645_2""",0,"""This is a review of 'Freddy Mercury The Untol...",6
17060,"""5920_9""",1,"""American icon Henry Fonda portrays \""Elegant\...",7
24744,"""9501_2""",0,"""This is the page for \""House of Exorcism\"", b...",8
5253,"""7877_4""",0,"""The makers of this film have created a future...",9


In [20]:
#append the true class value
probssentences["truelabel"] = 0
for i in range(len(probssentences)):
    #get true label value from test1 where index matches doc number
    j = probssentences.iloc[i]['doc']
    probssentences.set_value(i, 'truelabel', test1.iloc[[j]]["sentiment"])

In [6]:
probssentences[0:10]

,0,1,doc,truelabel
0,0.5986,0.4014,0,0
1,0.00792023,0.99208,0,0
2,0.369248,0.630752,0,0
3,0.445318,0.554682,0,0
4,0.999581,0.000419095,0,0
5,0.00449226,0.995508,1,0
6,0.918007,0.0819928,1,0
7,0.55223,0.44777,1,0
8,0.986808,0.0131918,1,0
9,0.721142,0.278858,1,0


In [21]:
#split test set into two
fracTrain = 0.5
nSamples = test1.shape[0]
print nSamples
order = np.random.permutation(nSamples) # come up with a random ordering
print len(order)
splitIndex = int(np.round(nSamples*fracTrain))
print len(order[:splitIndex])
print len(order[splitIndex:])
testA = test1.iloc[order[:splitIndex],:]
testB = test1.iloc[order[splitIndex:],:]

12500
12500
6250
6250


In [54]:
#order 2 test sets in order of docs?

In [22]:
#split probssentences the same way
probssentencesA = probssentences.loc[probssentences['doc'].isin(testA['doc'].unique())]
probssentencesB = probssentences.loc[probssentences['doc'].isin(testB['doc'].unique())]

In [23]:
print len(probssentences)
print len(probssentencesA)
print len(probssentencesB)
print len(testA)
print len(testB)

133515
66361
67154
6250
6250


In [24]:
probssentencesA[0:10]

,0,1,doc,truelabel
0,0.5986,0.4014,0,0
1,0.00792023,0.99208,0,0
2,0.369248,0.630752,0,0
3,0.445318,0.554682,0,0
4,0.999581,0.000419095,0,0
12,0.0193013,0.980699,2,1
13,0.0159692,0.984031,2,1
14,0.0224181,0.977582,2,1
15,0.986999,0.0130013,2,1
19,0.00138163,0.998618,4,0


In [25]:
probssentencesB[0:10]

,0,1,doc,truelabel
5,0.00449226,0.995508,1,0
6,0.918007,0.0819928,1,0
7,0.55223,0.44777,1,0
8,0.986808,0.0131918,1,0
9,0.721142,0.278858,1,0
10,0.111639,0.888361,1,0
11,0.717194,0.282805,1,0
16,0.0593864,0.940614,3,1
17,0.0795348,0.920465,3,1
18,0.00578835,0.994212,3,1


In [26]:
testA[0:10]

,id,sentiment,review,doc
12313,"""10709_1""",0,"""I can't think of much to say about this film....",1700
2757,"""2702_9""",1,"""OK, so I know of this movie because of a frie...",4823
23834,"""6490_10""",1,"""Still the definitive program about the Second...",8536
21756,"""5709_1""",0,"""Nick Cage is Gates, a treasure hunter (oh, ex...",5428
1164,"""1132_4""",0,"""The point of the vastly extended preparatory ...",7111
10508,"""8260_9""",1,"""Those French and those Germans sure have a lo...",4925
10295,"""4796_8""",1,"""\""A truly nice story with a moral about broth...",5786
15194,"""6833_10""",1,"""Beautiful film, pure Cassavetes style. Gena R...",8456
654,"""1113_10""",1,"""Sandra Bernhard is quite a character, and cer...",2508
18595,"""5054_7""",1,"""Richard Attenborough who already given us mag...",5858


In [27]:
#checking that testB matches probssentencesB
print testB.loc[testB['doc']==2]

Empty DataFrame
Columns: [id, sentiment, review, doc]
Index: []


### NOTE: probssentences and test set are not in same order. Must match on doc number.

In [28]:
print len(testA['doc'].unique())

6250


In [78]:
a = datetime.datetime.now().replace(microsecond=0)

#put into function and make split number the parameter
def getFeaturesAndLabels(numSplits):
    i = 0
    featuresNeg = []
    featuresPos = []
    labels = []
    #for each doc in probssentences:
    for doc in testA['doc'].unique():  
        avgScoresPos = []
        avgScoresNeg = []
        df = probssentencesA.loc[probssentencesA['doc'] == doc]
        if len(df) >= numSplits:
            #split into x groups
            lst = np.array_split(df, numSplits)
            for z in lst:
                #make the average score for each group the feature
                avgScoresNeg.append(z[[0]].mean()[0])
                avgScoresPos.append(z[[1]].mean()[1])
                #convert to a 0 or 1 (?)
            featuresNeg.append(avgScoresNeg)
            featuresPos.append(avgScoresPos)
            labels.append(testA.loc[testA['doc'] == doc]['sentiment'].values[0])
        i = i + 1
    return (featuresNeg, featuresPos, labels)
    
#function call
x = 3
tuple1 = getFeaturesAndLabels(x)

b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:01:28


In [82]:
print len(tuple1)

3


In [84]:
#ignore those reviews with fewer sentences than splits, so instead of 6250 datapoints to build on, have lost about 242:
print len(tuple1[0])
print len(tuple1[1])
print len(tuple1[2])
featuresPos = tuple1[0]
featuresNeg = tuple1[1]
labels = tuple1[2]

6018
6018
6018


In [85]:
#perform feature weighting-
clf = ExtraTreesClassifier()
clf = clf.fit(featuresNeg, labels)
weightsNeg = clf.feature_importances_  
print weightsNeg
clf = ExtraTreesClassifier()
clf = clf.fit(featuresPos, labels)
weightsPos = clf.feature_importances_
print weightsPos

[ 0.32137548  0.33372699  0.34489753]
[ 0.35465163  0.34982594  0.29552243]


### Split test set 2 into x subsets as well

In [88]:
a = datetime.datetime.now().replace(microsecond=0)

#numSplits variable to be consistent with training set
x = 3

i = 0
featuresTestPos = []
featuresTestNeg = []
labelsTest = []
#for each doc in probssentences:
for doc in testB['doc'].unique():
    avgScoresTestPos = []
    avgScoresTestNeg = []
    df = probssentencesB.loc[probssentencesB['doc'] == doc]
    if len(df) >= x:
        #split into x groups
        lst = np.array_split(df, x)
        for z in lst:
            avgScoresTestNeg.append(z[[0]].mean()[0])
            avgScoresTestPos.append(z[[1]].mean()[1])
        featuresTestNeg.append(avgScoresTestNeg)
        featuresTestPos.append(avgScoresTestPos)
        labelsTest.append(testB.loc[testB['doc'] == doc]['sentiment'].values[0])
    i = i + 1

b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:01:26


In [89]:
print (featuresTestPos*weightsPos).shape
print (featuresTestPos*weightsPos)[0]
print featuresTestPos[0]
print weightsPos

(6005, 3)
[ 0.22891767  0.1762777   0.16712046]
[0.64547192901372907, 0.50390118360519409, 0.56550852954387665]
[ 0.35465163  0.34982594  0.29552243]


In [90]:
print (featuresTestNeg*weightsNeg).shape
print (featuresTestNeg*weightsNeg)[0]
print featuresTestNeg[0]
print weightsNeg

(6005, 3)
[ 0.11393664  0.16556156  0.14985503]
[0.35452809333801272, 0.49609879404306412, 0.43449146300554276]
[ 0.32137548  0.33372699  0.34489753]


In [91]:
#get probs for each doc in test2:
probsThreePos = featuresTestPos * weightsPos
print probsThreePos.shape
#combine into one prob per review
probPos = probsThreePos.mean(1)
print probsThreePos[0]
print probPos[0]
print probPos.shape

(6005, 3)
[ 0.22891767  0.1762777   0.16712046]
0.190771943971
(6005,)


In [92]:
probsThreeNeg = featuresTestNeg * weightsNeg
print probsThreeNeg.shape
#combine into one prob per review
probNeg = probsThreeNeg.mean(1)
print probsThreeNeg[0]
print probNeg[0]
print probNeg.shape

(6005, 3)
[ 0.11393664  0.16556156  0.14985503]
0.143117741845
(6005,)


In [ ]:
#labelsTest is in same order as the doc probs

In [93]:
preds = np.ones((probPos.shape[0]))
print preds.shape

(6005,)


In [94]:
preds[np.where(probPos < probNeg)] = 0 # The first column is the negative model

In [97]:
print preds[0:10]
print probPos[0:10]
print probNeg[0:10]

[ 1.  0.  0.  1.  1.  0.  1.  1.  1.  0.]
[ 0.19077194  0.10215564  0.06243355  0.1744457   0.17002082  0.1197537
  0.22775357  0.21177292  0.22857614  0.12202001]
[ 0.14311774  0.22806665  0.27076661  0.16778775  0.1661257   0.21666086
  0.10850201  0.12447677  0.10833112  0.21852137]


In [106]:
print type(preds)
print preds.shape
#convert labels also to numpy array
labelsTest = np.asarray(labelsTest)
print labelsTest.shape

<type 'numpy.ndarray'>
(6005,)
(6005,)


In [107]:
print np.size(np.where(preds == labelsTest))*1./np.size(preds)

0.867610324729


In [108]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(labelsTest, preds)

0.86689992426


Essentially same performance. Why??

In [112]:
print test1['review'].iloc[0]

"This movie is like the thousand \"cat and mouse\" movies that preceded it. (The following may look like a spoiler, but it really just describes a large class of movies) There is the passionate, wise main character, his goofy but well-meaning sidekick with his ill-placed attempts at humorous comments, the initially-hostile but soon softened gorgeous lady who triggers the inevitable \"unlikely\" love story, the loved ones taken hostage, and of course the careless evil adversary with his brutal minions. Everybody has seen tons of these movies already, and \"National Treasure\" is like any one of them, with only a slightly modified wrapping. Every turn of the story was easily predicted (and I can assure you I am not the sharpest tool in the shed). I am quite tired of feeling tricked for money after exiting the theater from a Hollywood movie, and if you have ever felt that way too, heed my warning; stay miles away from this movie."


In [111]:
print test1[0:12]

              id  sentiment  \
21130   "5706_2"          0   
3404   "11132_3"          0   
14560  "10770_7"          1   
6671   "1058_10"          1   
10982  "12239_1"          0   
5526     "966_7"          1   
19301  "11645_2"          0   
17060   "5920_9"          1   
24744   "9501_2"          0   
5253    "7877_4"          0   
21967   "5442_8"          1   
18609   "7027_4"          0   

                                                  review  doc  
21130  "This movie is like the thousand \"cat and mou...    0  
3404   "A documentarist, like any filmmaker, must con...    1  
14560  "With a well thought out cast, this movie was ...    2  
6671   "this is the best sci-fi that I have seen in m...    3  
10982  "Ever since I was eight years old I have been ...    4  
5526   "2005 Toronto Film Festival Report It is offic...    5  
19301  "This is a review of 'Freddy Mercury The Untol...    6  
17060  "American icon Henry Fonda portrays \"Elegant\...    7  
24744  "This is the 